In [1]:
# Standard
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings; warnings.simplefilter('ignore')

# Machine Learning
from scipy import stats
import sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, precision_score, precision_recall_curve, recall_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

# NLP 
import string
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import RegexpTokenizer
# nltk.download('wordnet')
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
df1 = pd.read_csv(r"C:\Users\Ryan\Desktop\UCI\Spring Quarter\NLP\Final Project\data\earning_calls.csv")
df1.head(n=10)

,href,Transcript
0,/earnings/call-transcripts/2019/07/18/danaher-...,"[""OperatorGood morning. My name is Laurie, and..."
1,/earnings/call-transcripts/2019/05/01/vertex-p...,"['Prepared Remarks:', 'Michael Partridge -- Se..."
2,/earnings/call-transcripts/2018/04/24/3m-compa...,"['Prepared Remarks:', 'Operator', ""Thank you f..."
3,/earnings/call-transcripts/2018/11/16/nvidia-c...,"['Contents:', '', 'Prepared Remarks', 'Questio..."
4,/earnings/call-transcripts/2018/01/15/jpmorgan...,"['Prepared Remarks:', 'Operator', ""Good mornin..."
5,/earnings/call-transcripts/2018/07/31/apple-in...,"['Prepared Remarks:', 'Operator', ""Good day, a..."
6,/earnings/call-transcripts/2020/04/30/microsof...,"[""OperatorGreetings and welcome to the Microso..."
7,/earnings/call-transcripts/2018/02/01/amazonco...,"['Prepared Remarks:', 'Operator', ""Thank you f..."
8,/earnings/call-transcripts/2018/10/31/anthem-i...,"['Prepared Remarks:', 'Operator', 'Ladies and ..."
9,/earnings/call-transcripts/2018/07/17/unitedhe...,"['Prepared Remarks:', 'Operator', ""Good mornin..."


In [3]:
# Set index
df1['Index'] = range(1, len(df1)+1)
df1.set_index('Index')

,href,Transcript
Index,,
1,/earnings/call-transcripts/2019/07/18/danaher-...,"[""OperatorGood morning. My name is Laurie, and..."
2,/earnings/call-transcripts/2019/05/01/vertex-p...,"['Prepared Remarks:', 'Michael Partridge -- Se..."
3,/earnings/call-transcripts/2018/04/24/3m-compa...,"['Prepared Remarks:', 'Operator', ""Thank you f..."
4,/earnings/call-transcripts/2018/11/16/nvidia-c...,"['Contents:', '', 'Prepared Remarks', 'Questio..."
5,/earnings/call-transcripts/2018/01/15/jpmorgan...,"['Prepared Remarks:', 'Operator', ""Good mornin..."
...,...,...
800,/earnings/call-transcripts/2020/05/07/bristol-...,"[""OperatorGood day and welcome to the BristolM..."
801,/earnings/call-transcripts/2018/02/02/merck-co...,"['Prepared Remarks:', 'Operator', ""Good mornin..."
802,/earnings/call-transcripts/2019/01/30/amgen-in...,"['Contents:', '', 'Prepared Remarks', 'Questio..."


In [4]:
# Get Date

# Split href
href = df1['href'].str.split("/",n=6,expand=True)

# Drop unnecessary axes
href1 = href.drop([0,1,2,6], axis = 1)

# Rename Axes
href1 = href1.rename(columns = {3:"Year",4:"Month",5:"Day"})
href1['Date'] = pd.to_datetime(href1[['Year', 'Month', 'Day']])

# Create Index to merge on
href1['Index'] = range(1, len(href1)+1)
href1.set_index('Index')
href1.head()

# Join to Original Dataframe 
df1 = df1.merge(href1, how = 'outer', on = 'Index' )
df1.head()

,href,Transcript,Index,Year,Month,Day,Date
0,/earnings/call-transcripts/2019/07/18/danaher-...,"[""OperatorGood morning. My name is Laurie, and...",1,2019,07,18,2019-07-18
1,/earnings/call-transcripts/2019/05/01/vertex-p...,"['Prepared Remarks:', 'Michael Partridge -- Se...",2,2019,05,01,2019-05-01
2,/earnings/call-transcripts/2018/04/24/3m-compa...,"['Prepared Remarks:', 'Operator', ""Thank you f...",3,2018,04,24,2018-04-24
3,/earnings/call-transcripts/2018/11/16/nvidia-c...,"['Contents:', '', 'Prepared Remarks', 'Questio...",4,2018,11,16,2018-11-16
4,/earnings/call-transcripts/2018/01/15/jpmorgan...,"['Prepared Remarks:', 'Operator', ""Good mornin...",5,2018,01,15,2018-01-15


In [5]:
# Get Quarter

# Split existing href
href2 = href[6].str.split(r'-\d',n = 3, expand = True)
href3 = href2[0].str.split('-q', n = 4, expand = True)

# Rename and drop unnessecary columns
href3 = href3.rename(columns = {1:'Quarter'})
href3 = href3.drop(columns = [0,2])

# Create Index
href3['Index'] = range(1, len(href1)+1)
href3.set_index('Index')
href3.head()

# Merge to existing dataframe
df1 = df1.merge(href3, how = 'outer', on = 'Index')
df1.head()

,href,Transcript,Index,Year,Month,Day,Date,Quarter
0,/earnings/call-transcripts/2019/07/18/danaher-...,"[""OperatorGood morning. My name is Laurie, and...",1,2019,07,18,2019-07-18,2
1,/earnings/call-transcripts/2019/05/01/vertex-p...,"['Prepared Remarks:', 'Michael Partridge -- Se...",2,2019,05,01,2019-05-01,1
2,/earnings/call-transcripts/2018/04/24/3m-compa...,"['Prepared Remarks:', 'Operator', ""Thank you f...",3,2018,04,24,2018-04-24,1
3,/earnings/call-transcripts/2018/11/16/nvidia-c...,"['Contents:', '', 'Prepared Remarks', 'Questio...",4,2018,11,16,2018-11-16,3
4,/earnings/call-transcripts/2018/01/15/jpmorgan...,"['Prepared Remarks:', 'Operator', ""Good mornin...",5,2018,01,15,2018-01-15,4


In [6]:
#bootleg company name column
df1['CompanyName'] = df1['href'].str[38:]

In [7]:
# Reorder column names

cols = df1.columns.tolist()
myorder = [7,0,5,6,1,3,4,2]
cols = [cols[i] for i in myorder]
df1 = df1[cols]

In [8]:
df1.head()

,Quarter,href,Day,Date,Transcript,Year,Month,Index
0,2,/earnings/call-transcripts/2019/07/18/danaher-...,18,2019-07-18,"[""OperatorGood morning. My name is Laurie, and...",2019,07,1
1,1,/earnings/call-transcripts/2019/05/01/vertex-p...,01,2019-05-01,"['Prepared Remarks:', 'Michael Partridge -- Se...",2019,05,2
2,1,/earnings/call-transcripts/2018/04/24/3m-compa...,24,2018-04-24,"['Prepared Remarks:', 'Operator', ""Thank you f...",2018,04,3
3,3,/earnings/call-transcripts/2018/11/16/nvidia-c...,16,2018-11-16,"['Contents:', '', 'Prepared Remarks', 'Questio...",2018,11,4
4,4,/earnings/call-transcripts/2018/01/15/jpmorgan...,15,2018-01-15,"['Prepared Remarks:', 'Operator', ""Good mornin...",2018,01,5


# Ryan's Sandbox

In [9]:
import re

In [10]:
# Contraction dictionary
contractions_dict = {
    "didn't": 'did not',
    "don't": 'do not',
    "aren't": 'are not',
    "can't": 'cannot',
    "could've": "could've",
    "couldn't": "could not",
    "i'll": "i will",
    "i'd": "i would",
    "i'm": "i am",
    "it'll": "it will",
    "we'll": "we will"
    
}

contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))

def expand_contractions(s, contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)

In [11]:
expand_contractions(r"couldn't")

'could not'

In [12]:
# Standard function to clean string
def clean(string):
    x = string.lower() # lowercases the string
    x = expand_contractions(x) # replaces contractions
    x = re.sub(r'\W+', ' ', x) # takes only alpha numeric
    return x

In [13]:
df1['clean_transcript'] = df1.Transcript.apply(lambda x: clean(x))

In [14]:
df1.head()

,Quarter,href,Day,Date,Transcript,Year,Month,Index,clean_transcript
0,2,/earnings/call-transcripts/2019/07/18/danaher-...,18,2019-07-18,"[""OperatorGood morning. My name is Laurie, and...",2019,07,1,operatorgood morning my name is laurie and i ...
1,1,/earnings/call-transcripts/2019/05/01/vertex-p...,01,2019-05-01,"['Prepared Remarks:', 'Michael Partridge -- Se...",2019,05,2,prepared remarks michael partridge senior vic...
2,1,/earnings/call-transcripts/2018/04/24/3m-compa...,24,2018-04-24,"['Prepared Remarks:', 'Operator', ""Thank you f...",2018,04,3,prepared remarks operator thank you for stand...
3,3,/earnings/call-transcripts/2018/11/16/nvidia-c...,16,2018-11-16,"['Contents:', '', 'Prepared Remarks', 'Questio...",2018,11,4,contents prepared remarks questions and answe...
4,4,/earnings/call-transcripts/2018/01/15/jpmorgan...,15,2018-01-15,"['Prepared Remarks:', 'Operator', ""Good mornin...",2018,01,5,prepared remarks operator good morning ladies...


In [15]:
# List of stopwords
stop = list(STOPWORDS)

# Functions
# removes stop words from a clean transcript
def remove_stop(string):
    wostop = [] # empty list to append to
    word = string.split() # splits string into list
    for n, i in enumerate(word): # iterates over the list
        if i not in stop: # if word in the list not in stop words list
            wostop.append(word[n]) # append to wostop list
    wostop = ' '.join(wostop) # join wostop list into a string
    return wostop # returns a string

# Stems word
def stem(string):
    t = [] # empty list to append
    ps = nltk.stem.PorterStemmer() # stem purposes
    word = string.split() # split string into list
    for item in word: # iterate through list
        stem = ps.stem(item) # stem each word
        t.append(stem) # add new stem word into list
    t = ' '.join(t) # join list of stem words into a string
    return t # returns a string

In [16]:
df1['clean_transcript2'] = df1.clean_transcript.apply(lambda x: stem(remove_stop(x)))
df1

,Quarter,href,Day,Date,Transcript,Year,Month,Index,clean_transcript,clean_transcript2
0,2,/earnings/call-transcripts/2019/07/18/danaher-...,18,2019-07-18,"[""OperatorGood morning. My name is Laurie, and...",2019,07,1,operatorgood morning my name is laurie and i ...,operatorgood morn name lauri will confer facil...
1,1,/earnings/call-transcripts/2019/05/01/vertex-p...,01,2019-05-01,"['Prepared Remarks:', 'Michael Partridge -- Se...",2019,05,2,prepared remarks michael partridge senior vic...,prepar remark michael partridg senior vice pre...
2,1,/earnings/call-transcripts/2018/04/24/3m-compa...,24,2018-04-24,"['Prepared Remarks:', 'Operator', ""Thank you f...",2018,04,3,prepared remarks operator thank you for stand...,prepar remark oper thank stand welcom 3m first...
3,3,/earnings/call-transcripts/2018/11/16/nvidia-c...,16,2018-11-16,"['Contents:', '', 'Prepared Remarks', 'Questio...",2018,11,4,contents prepared remarks questions and answe...,content prepar remark question answer call par...
4,4,/earnings/call-transcripts/2018/01/15/jpmorgan...,15,2018-01-15,"['Prepared Remarks:', 'Operator', ""Good mornin...",2018,01,5,prepared remarks operator good morning ladies...,prepar remark oper good morn ladi gentlemen we...
...,...,...,...,...,...,...,...,...,...,...
799,1,/earnings/call-transcripts/2020/05/07/bristol-...,07,2020-05-07,"[""OperatorGood day and welcome to the BristolM...",2020,05,800,operatorgood day and welcome to the bristolmy...,operatorgood day welcom bristolmy squibb 2020 ...
800,4,/earnings/call-transcripts/2018/02/02/merck-co...,02,2018-02-02,"['Prepared Remarks:', 'Operator', ""Good mornin...",2018,02,801,prepared remarks operator good morning my nam...,prepar remark oper good morn name darla will c...
801,4,/earnings/call-transcripts/2019/01/30/amgen-in...,30,2019-01-30,"['Contents:', '', 'Prepared Remarks', 'Questio...",2019,01,802,contents prepared remarks questions and answe...,content prepar remark question answer call par...
802,3,/earnings/call-transcripts/2017/10/16/wells-fa...,16,2017-10-16,"['Prepared Remarks:', 'Operator', ""Good mornin...",2017,10,803,prepared remarks operator good morning my nam...,prepar remark oper good morn name regina will ...


In [26]:
test = df1.href[0]
test = test.split('/')
test = test[6]
test.replace('-', ' ')

'danaher corp dhr q2 2019 earnings call transcript'

# End of Ryan's Sand Box

In [36]:
# Stemmer

stemmer = PorterStemmer()

def word_stemmer(text):
    stem_text = ''.join([stemmer.stem(i) for i in text])
    return stem_text

df1['Transcript'] = df1['Transcript'].apply(lambda x: word_stemmer(x))
# df1.head()

In [37]:
#do some text analysis and visualization w/ wordcloud
df1['Transcript'] = df1['Transcript'].apply(str)

In [38]:
#convert all text to lower
df1['Transcript'] = df1['Transcript'].str.lower()

In [39]:
stopwords = set(STOPWORDS)
#stopwords.update(["words in here"])

In [40]:
#removes all stopwords, update in above cell as you wish
df1['Transcript'] = df1['Transcript'].apply(lambda x: ''.join([word for word in x.split() if word not in (stopwords)]))

In [41]:
#Remove punctuation
df1["Transcript"] = df1['Transcript'].str.replace('[^\w\s]','')

In [42]:
#clean href column
#df1["Company_Name"] = df1['href'].str.replace('/earnings/call-transcripts/','')

In [43]:
df1.head(n=10)

,Index,Quarter_x,Year,Month,href,Date,Transcript,Day
0,1,2,2019,07,/earnings/call-transcripts/2019/07/18/danaher-...,2019-07-18,opetogoodmonngmynmesluendllbeyouconfeencefcltt...,18
1,2,1,2019,05,/earnings/call-transcripts/2019/05/01/vertex-p...,2019-05-01,pepedemsmchelptdgesenovcepesdentofnvestoeltons...,01
2,3,1,2018,04,/earnings/call-transcripts/2018/04/24/3m-compa...,2018-04-24,pepedemsopetothnyoufostndngbywelcometothe3mfst...,24
3,4,3,2018,11,/earnings/call-transcripts/2018/11/16/nvidia-c...,2018-11-16,contentspepedemsquestonsndnswescllptcpntspeped...,16
4,5,4,2018,01,/earnings/call-transcripts/2018/01/15/jpmorgan...,2018-01-15,pepedemsopetogoodmonngldesndgentlemenwelcometo...,15
5,6,3,2018,07,/earnings/call-transcripts/2018/07/31/apple-in...,2018-07-31,pepedemsopetogooddyndwelcometothepplencopotedt...,31
6,7,3,2020,04,/earnings/call-transcripts/2020/04/30/microsof...,2020-04-30,opetogeetngsndwelcometothemcosoftfsclye2020thd...,30
7,8,4,2018,02,/earnings/call-transcripts/2018/02/01/amazonco...,2018-02-01,pepedemsopetothnyoufostndngbygooddyeveyonendwe...,01
8,9,3,2018,10,/earnings/call-transcripts/2018/10/31/anthem-i...,2018-10-31,pepedemsopetoldesndgentlementhnyoufostndngbywe...,31
9,10,2,2018,07,/earnings/call-transcripts/2018/07/17/unitedhe...,2018-07-17,pepedemsopetogoodmonngwllbeyouconfeenceopetoto...,17


In [44]:
#this joins all cell values into one string
text1 = " ".join(word for word in df1.Transcript)

In [ ]:

#create stopword list
stopwords = set(STOPWORDS)
stopwords.update(["vice", "president", "chief", "executive", "financial", "officer"])

#resize/make it bigger
plt.subplots(figsize = (25,15))

#This generates the word cloud
cloud_text = WordCloud(stopwords = stopwords, background_color = "black").generate(text1)

# This creates a figure of the generated cloud
plt.imshow(cloud_text, interpolation='bilinear')  
plt.axis('off')
# Display the figure
plt.show()

In [ ]:
#this function will display word count frequency
#basically we can have a word cloud, but an actual number associated with it
def word_freq(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
word_freq(df1.Transcript)

In [ ]:
train, test = train_test_split(df1, test_size = 0.2)

train.head()
test.head()

In [ ]:
unigram = CountVectorizer(ngram_range=(1,1))
bigram = CountVectorizer(ngram_range=(2,2))
trigram = CountVectorizer(ngram_range=(3,3))
#fourgram = CountVectorizer(ngram_range=(4,4))
combined = CountVectorizer(ngram_range=(1,3))
vectorizers = [unigram, bigram, trigram, combined]
print("Fitting vectorizers")
[vectorizer.fit(df1.Transcript) for vectorizer in vectorizers]

In [ ]:
def show_distribution(vectorizer, train):
  vecs = vectorizer.transform(df1.Transcript)
  sum_mat = np.sum(vecs, axis=0)
  freqs = np.sort(sum_mat).T[::-1]
  plt.plot(list(range(1, sum_mat.shape[1] + 1)), freqs)
  plt.xlabel('Rank')
  plt.ylabel('Frequency')
  plt.show()
  
  plt.loglog(list(range(1, sum_mat.shape[1] + 1)), freqs)
  plt.xlabel('Rank')
  plt.ylabel('Frequency')
  plt.show()

In [ ]:
show_distribution(unigram, train)

In [ ]:
def ngram2idx(ngram, vocab_dict):
  index = vocab_dict[ngram] if ngram in vocab_dict.keys() else 'Not Found'
  print(ngram, ' -> ', index)


def idx2ngram(index, vocabs):
  ngram = vocabs[index] if 0 <= index < len(vocabs) else 'Not Found'
  print(index, ' -> ', ngram)

In [ ]:
vectorizer = unigram # change to bigram or trigram
vocab_dict = vectorizer.vocabulary_
vocabs = vectorizer.get_feature_names()


In [ ]:
def sentence2vec(sentence, vectorizer):
  vec = vectorizer.transform([sentence])
  vocabs = vectorizer.get_feature_names()
  print('\"', sentence, '\" -> ')
  print(vec)
  for idx in vec.indices:
    print(idx, vocabs[idx])
  print()

In [ ]:
tfidf = TfidfVectorizer()
tfidf.fit(train.Transcript)

In [ ]:
X = tfidf.transform(train.Transcript)
print(X.toarray())

In [ ]:
tfidf_cos = cosine_similarity(X)
tfidf_cos

In [ ]:
bog =  CountVectorizer()
bog.fit(train.Transcript)

In [ ]:
Y = bog.transform(train.Transcript)

In [ ]:
bog_cos = cosine_similarity(Y)
bog_cos

In [ ]:
tfidf_bigram = TfidfVectorizer(ngram_range=(2,2))
tfidf_trigram = TfidfVectorizer(ngram_range=(3,3))
tfidf_combined = TfidfVectorizer(ngram_range=(1,3))
tfidf_bigram.fit(train.Transcript)
tfidf_trigram.fit(train.Transcript)
tfidf_combined.fit(train.Transcript)
tfidf_vectorizers = [tfidf, tfidf_bigram, tfidf_trigram, tfidf_combined]

In [ ]:
show_distribution(tfidf_bigram, train)

In [ ]:
tfidf = TfidfVectorizer(analyzer = 'word', ngram_range=(1,2),stop_words = 'english')

In [ ]:
#this is similar to the movie recommender
tfidf_matrix = tfidf.fit_transform(train['Transcript'])

In [ ]:
#similar to movie recommender
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
df1 = df1.reset_index()
earn_call = df1['CompanyName']
indices = pd.Series(df1.index, index = df1['CompanyName'])

In [ ]:
def get_recommendations(call):
    idx = indices[call]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    website_indices = [i[0] for i in sim_scores]
    return earn_call.iloc[website_indices]

In [ ]:
#this shows earnings calls that are similar to each other in terms of text
get_recommendations('danaher-corp-dhr-q2-2019-earnings-call-transcript').head(10)

In [ ]:
print(df1.CompanyName[1])

### Sentiment Analysis

In [ ]:
#vader = SentimentIntensityAnalyzer()
#temp3 = vader.polarity_scores(df1['Transcript'])

In [ ]:
#temp3